# 3D Candlestick Visualization for BTC

This notebook demonstrates how to visualize 4-dimensional candlestick data (opening, closing, upper wick, lower wick) in a 3D interactive plot for Bitcoin daily data.

In [40]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import plotly.express as px

In [41]:
# Configuration dictionary
config = {
    'ticker': 'BTC-USD',
    'period': '1y',        # 1 year of data
    'interval': '1d',      # daily data
    'plot_title': 'BTC Daily Candlestick 3D Visualization',
    'width': 1000,         # plot width
    'height': 800,         # plot height
    'bull_color': 'green', # color for rising candles
    'bear_color': 'red',   # color for falling candles
    'opacity': 0.7,        # transparency of candles
    'show_volume': True    # whether to include volume as marker size
}

In [42]:
# Fetch BTC data
def get_btc_data(config):
    ticker = yf.Ticker(config['ticker'])
    data = ticker.history(period=config['period'], interval=config['interval'])
    return data

# Get the data
btc_data = get_btc_data(config)
btc_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-03-13 00:00:00+00:00,71482.117188,73637.476562,71334.093750,73083.500000,48212536929,0.0,0.0
2024-03-14 00:00:00+00:00,73079.375000,73750.070312,68563.023438,71396.593750,59594605698,0.0,0.0
2024-03-15 00:00:00+00:00,71387.875000,72357.132812,65630.695312,69403.773438,78320453976,0.0,0.0
2024-03-16 00:00:00+00:00,69392.484375,70046.273438,64801.394531,65315.117188,46842198371,0.0,0.0
2024-03-17 00:00:00+00:00,65316.343750,68845.718750,64545.316406,68390.625000,44716864318,0.0,0.0


In [43]:
# Display basic statistics of the data
btc_data.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,365.000000,365.000000,365.000000,365.000000,3.650000e+02,365.0,365.0
mean,74940.126306,76436.080041,73351.737457,74971.495676,4.091917e+10,0.0,0.0
std,15734.040050,16036.429145,15396.733874,15737.811503,2.317093e+10,0.0,0.0
min,53949.085938,54838.144531,49121.238281,53948.753906,9.858199e+09,0.0,0.0
25%,62941.425781,64125.687500,61689.582031,62940.457031,2.617526e+10,0.0,0.0
50%,67840.570312,68969.750000,66611.296875,67837.640625,3.487353e+10,0.0,0.0
75%,93527.195312,95174.875000,91371.742188,93530.226562,4.908432e+10,0.0,0.0
max,106147.296875,109114.882812,105291.734375,106146.265625,1.492189e+11,0.0,0.0


In [44]:
# Prepare data for 3D visualization
def prepare_data_for_3d(df):
    # Create a numerical index for x-axis (days)
    df['day_index'] = range(len(df))
    
    # Calculate wick lengths
    df['upper_wick'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['lower_wick'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    
    # Determine candle color
    df['is_bull'] = df['Close'] >= df['Open']
    
    return df

# Prepare the data
prepared_data = prepare_data_for_3d(btc_data.copy())
prepared_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,day_index,upper_wick,lower_wick,is_bull
Date,,,,,,,,,,,
2024-03-13 00:00:00+00:00,71482.117188,73637.476562,71334.093750,73083.500000,48212536929,0.0,0.0,0,553.976562,148.023438,True
2024-03-14 00:00:00+00:00,73079.375000,73750.070312,68563.023438,71396.593750,59594605698,0.0,0.0,1,670.695312,2833.570312,False
2024-03-15 00:00:00+00:00,71387.875000,72357.132812,65630.695312,69403.773438,78320453976,0.0,0.0,2,969.257812,3773.078125,False
2024-03-16 00:00:00+00:00,69392.484375,70046.273438,64801.394531,65315.117188,46842198371,0.0,0.0,3,653.789062,513.722656,False
2024-03-17 00:00:00+00:00,65316.343750,68845.718750,64545.316406,68390.625000,44716864318,0.0,0.0,4,455.093750,771.027344,True


In [45]:
# Create 3D interactive visualization
def plot_3d_candlestick(df, config):
    # Create figure
    fig = go.Figure()
    
    # Calculate marker size based on volume if enabled
    if config['show_volume']:
        marker_size = 50 * (df['Volume'] / df['Volume'].max())
    else:
        marker_size = 20
    
    # Create scatter3d trace for bullish candles
    bull_data = df[df['is_bull']]
    if not bull_data.empty:
        fig.add_trace(go.Scatter3d(
            x=bull_data['day_index'],
            y=bull_data['Open'],
            z=bull_data['Close'],
            mode='markers',
            marker=dict(
                size=marker_size.loc[bull_data.index],
                color=config['bull_color'],
                opacity=config['opacity'],
            ),
            name='Bullish Candles',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{y:.2f}<br>'
                'Close: %{z:.2f}<br>'
                'High: %{customdata[0]:.2f}<br>'
                'Low: %{customdata[1]:.2f}<br>'
                'Upper Wick: %{customdata[2]:.2f}<br>'
                'Lower Wick: %{customdata[3]:.2f}'
            ),
            text=bull_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bull_data['High'], bull_data['Low'], 
                                        bull_data['upper_wick'], bull_data['lower_wick']))
        ))
    
    # Create scatter3d trace for bearish candles
    bear_data = df[~df['is_bull']]
    if not bear_data.empty:
        fig.add_trace(go.Scatter3d(
            x=bear_data['day_index'],
            y=bear_data['Open'],
            z=bear_data['Close'],
            mode='markers',
            marker=dict(
                size=marker_size.loc[bear_data.index],
                color=config['bear_color'],
                opacity=config['opacity'],
            ),
            name='Bearish Candles',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{y:.2f}<br>'
                'Close: %{z:.2f}<br>'
                'High: %{customdata[0]:.2f}<br>'
                'Low: %{customdata[1]:.2f}<br>'
                'Upper Wick: %{customdata[2]:.2f}<br>'
                'Lower Wick: %{customdata[3]:.2f}'
            ),
            text=bear_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bear_data['High'], bear_data['Low'], 
                                        bear_data['upper_wick'], bear_data['lower_wick']))
        ))
    
    # Add line connecting points in time sequence
    fig.add_trace(go.Scatter3d(
        x=df['day_index'],
        y=df['Close'],
        z=df['Close'] * 0.99,  # Slightly offset to make it visible
        mode='lines',
        line=dict(color='grey', width=2),
        opacity=0.3,
        showlegend=False
    ))
    
    # Add wick lines (vertical lines for each candle representing high and low)
    for i, row in df.iterrows():
        # Upper wick
        if row['upper_wick'] > 0:
            fig.add_trace(go.Scatter3d(
                x=[row['day_index'], row['day_index']],
                y=[max(row['Open'], row['Close']), row['High']],
                z=[max(row['Open'], row['Close']), row['High']],
                mode='lines',
                line=dict(color='black', width=1),
                opacity=0.5,
                showlegend=False
            ))
        # Lower wick
        if row['lower_wick'] > 0:
            fig.add_trace(go.Scatter3d(
                x=[row['day_index'], row['day_index']],
                y=[min(row['Open'], row['Close']), row['Low']],
                z=[min(row['Open'], row['Close']), row['Low']],
                mode='lines',
                line=dict(color='black', width=1),
                opacity=0.5,
                showlegend=False
            ))
    
    # Update layout for better visualization
    fig.update_layout(
        title=config['plot_title'],
        scene=dict(
            xaxis_title='Time (Days)',
            yaxis_title='Open Price ($)',
            zaxis_title='Close Price ($)',
            aspectmode='auto'
        ),
        width=config['width'],
        height=config['height'],
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(x=0, y=0.9),
        template='plotly_white'
    )
    
    # Add annotations for key points
    max_price_idx = df['High'].idxmax()
    min_price_idx = df['Low'].idxmin()
    
    fig.add_trace(go.Scatter3d(
        x=[df.loc[max_price_idx, 'day_index']],
        y=[df.loc[max_price_idx, 'High']],
        z=[df.loc[max_price_idx, 'High']],
        mode='markers+text',
        text=['Highest Price'],
        marker=dict(size=10, color='gold'),
        name='Highest Price'
    ))
    
    fig.add_trace(go.Scatter3d(
        x=[df.loc[min_price_idx, 'day_index']],
        y=[df.loc[min_price_idx, 'Low']],
        z=[df.loc[min_price_idx, 'Low']],
        mode='markers+text',
        text=['Lowest Price'],
        marker=dict(size=10, color='blue'),
        name='Lowest Price'
    ))
    
    return fig

# Create and display the 3D candlestick plot
fig = plot_3d_candlestick(prepared_data, config)
fig.show()

## Alternative Visualization

We can also create another 3D visualization where the Z-axis represents the upper/lower wick lengths directly.

In [46]:
# Alternative 3D visualization with wick lengths as Z-axis
def plot_alternative_3d_candlestick(df, config):
    # Create figure
    fig = go.Figure()
    
    # Calculate marker size based on volume if enabled
    if config['show_volume']:
        marker_size = 40 * (df['Volume'] / df['Volume'].max())
    else:
        marker_size = 15
    
    # Create scatter3d trace for bullish candles
    bull_data = df[df['is_bull']]
    if not bull_data.empty:
        fig.add_trace(go.Scatter3d(
            x=bull_data['day_index'],
            y=(bull_data['Close'] + bull_data['Open']) / 2,  # Price level (midpoint)
            z=bull_data['upper_wick'],  # Upper wick length
            mode='markers',
            marker=dict(
                size=marker_size.loc[bull_data.index],
                color=config['bull_color'],
                opacity=config['opacity'],
                symbol='circle'
            ),
            name='Bullish Upper Wick',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{customdata[0]:.2f}<br>'
                'Close: %{customdata[1]:.2f}<br>'
                'Upper Wick: %{z:.2f}'
            ),
            text=bull_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bull_data['Open'], bull_data['Close']))
        ))
        
        fig.add_trace(go.Scatter3d(
            x=bull_data['day_index'],
            y=(bull_data['Close'] + bull_data['Open']) / 2,  # Price level (midpoint)
            z=-bull_data['lower_wick'],  # Negative for lower wick
            mode='markers',
            marker=dict(
                size=marker_size.loc[bull_data.index],
                color=config['bull_color'],
                opacity=config['opacity'],
                symbol='circle-open'
            ),
            name='Bullish Lower Wick',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{customdata[0]:.2f}<br>'
                'Close: %{customdata[1]:.2f}<br>'
                'Lower Wick: %{customdata[2]:.2f}'
            ),
            text=bull_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bull_data['Open'], bull_data['Close'], bull_data['lower_wick']))
        ))
    
    # Create scatter3d trace for bearish candles
    bear_data = df[~df['is_bull']]
    if not bear_data.empty:
        fig.add_trace(go.Scatter3d(
            x=bear_data['day_index'],
            y=(bear_data['Close'] + bear_data['Open']) / 2,  # Price level (midpoint)
            z=bear_data['upper_wick'],  # Upper wick length
            mode='markers',
            marker=dict(
                size=marker_size.loc[bear_data.index],
                color=config['bear_color'],
                opacity=config['opacity'],
                symbol='circle'
            ),
            name='Bearish Upper Wick',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{customdata[0]:.2f}<br>'
                'Close: %{customdata[1]:.2f}<br>'
                'Upper Wick: %{z:.2f}'
            ),
            text=bear_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bear_data['Open'], bear_data['Close']))
        ))
        
        fig.add_trace(go.Scatter3d(
            x=bear_data['day_index'],
            y=(bear_data['Close'] + bear_data['Open']) / 2,  # Price level (midpoint)
            z=-bear_data['lower_wick'],  # Negative for lower wick
            mode='markers',
            marker=dict(
                size=marker_size.loc[bear_data.index],
                color=config['bear_color'],
                opacity=config['opacity'],
                symbol='circle-open'
            ),
            name='Bearish Lower Wick',
            hovertemplate=(
                'Date: %{text}<br>'
                'Open: %{customdata[0]:.2f}<br>'
                'Close: %{customdata[1]:.2f}<br>'
                'Lower Wick: %{customdata[2]:.2f}'
            ),
            text=bear_data.index.strftime('%Y-%m-%d'),
            customdata=np.column_stack((bear_data['Open'], bear_data['Close'], bear_data['lower_wick']))
        ))
    
    # Connect candles with a line to show price movement over time
    fig.add_trace(go.Scatter3d(
        x=df['day_index'],
        y=(df['Open'] + df['Close']) / 2,  # Using midpoint
        z=df['upper_wick'] * 0,  # At z=0 plane
        mode='lines',
        line=dict(color='gray', width=2),
        opacity=0.5,
        name='Price Path'
    ))
    
    # Update layout
    fig.update_layout(
        title=config['plot_title'] + ' (Alternative View)',
        scene=dict(
            xaxis_title='Time (Days)',
            yaxis_title='Price Level ($)',
            zaxis_title='Wick Length (+ upper / - lower)',
            aspectmode='auto',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=0.8)
            )
        ),
        width=config['width'],
        height=config['height'],
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(x=0, y=0.9),
        template='plotly_white'
    )
    
    return fig

# Create and display the alternative 3D visualization
alt_fig = plot_alternative_3d_candlestick(prepared_data, config)
alt_fig.show()

## Conclusion

These visualizations provide interactive 3D representations of the 4-dimensional candlestick data for Bitcoin. The first visualization shows the traditional candlestick elements in 3D space, while the alternative visualization focuses on the relationship between price levels and wick lengths.

You can interact with these plots by:
- Rotating to view from different angles
- Zooming in/out
- Hovering over data points to see specific values
- Using the legend to filter specific components